In [3]:
from sqlalchemy.types import Date, DateTime, Integer, NVARCHAR
from sqlalchemy.sql import text
from sqlalchemy import create_engine
from urllib.parse import quote_plus

import pandas as pd


engine = create_engine('sqlite:///stocks.db', echo=False)
# sqlite_connection = engine.connect()

# conn = st.experimental_connection('stocks_db', type='sql')

df1 = pd.DataFrame({'name' : ['User 4', 'User 5', 'Usse 6']})
df1.to_sql(name='users', con=engine, if_exists='append')

# users = conn.query('select * from users')


3

In [1]:
from src.get_data import stock_wide_format, get_historical_price
import importlib
# importlib.reload(src.get_data)

stock_list = ['CTG', 'VCB', 'HPG', 'CTD', 'VIC', 'SAB', 'MWG','FPT','FRT'] 
# stocks_df_pivot = stock_wide_format(stock_list) 


In [3]:
vcb = get_historical_price('VCB')

2023-09-08 16:35:54,186 : INFO : data VND from 2018-01-10 to 2023-09-08 have already cloned!


In [11]:
vcb
vcb.droplevel('Symbols', axis=1)

Attributes,high,low,open,close,avg,volume
date,,,,,,
2018-01-10,27.75,27.10,27.55,27.50,27.520000,1382780.0
2018-01-11,27.50,26.80,27.30,27.20,27.210000,1260720.0
2018-01-12,28.20,27.35,27.45,27.60,27.760000,1730170.0
2018-01-15,28.40,27.35,27.60,28.25,28.110000,1273740.0
2018-01-16,28.40,27.90,28.10,28.25,28.140000,1077350.0
...,...,...,...,...,...,...
2023-08-31,23.80,23.30,23.55,23.50,23.510000,22510400.0
2023-09-05,23.85,23.30,23.70,23.50,23.530000,28464400.0
2023-09-06,24.45,23.20,23.50,24.00,23.950000,26487600.0


In [ ]:
stocks_df_pivot

Symbols,CTG,VCB,HPG,CTD,VIC,SAB,MWG,FPT,FRT
date,,,,,,,,,
2020-10-01,26.60,84.8,27.00,69.0,92.5,184.5,105.0,50.8,24.20
2020-10-02,26.95,84.0,26.70,68.7,94.0,184.5,104.0,50.3,24.10
2020-10-05,27.50,83.5,27.35,67.0,93.3,184.5,104.0,50.5,24.05
2020-10-06,27.20,83.9,28.25,64.5,91.8,186.3,105.2,50.5,24.45
2020-10-07,27.00,85.0,27.80,60.5,92.5,190.0,107.6,50.4,25.05
...,...,...,...,...,...,...,...,...,...
2023-08-30,32.10,89.1,27.20,61.7,62.0,159.8,53.8,96.6,83.30
2023-08-31,32.45,89.1,27.60,63.1,62.1,158.0,53.8,96.7,83.50
2023-09-05,32.65,90.1,27.80,65.3,62.3,158.8,54.3,96.8,82.60


In [ ]:
stocks_df_pivot.to_sql(name='stock_price_pivot', con=engine, if_exists='replace',
 dtype={
        'date': Date()
    })


733

In [ ]:

stocks_df_pivot = pd.read_sql('''select * from stock_price_pivot;''', con=engine)
stocks_df_pivot['date'] = pd.to_datetime(stocks_df_pivot['date'], format='%Y-%m-%d')
stocks_df_pivot.set_index('date', inplace=True)
stocks_df_pivot

,CTG,VCB,HPG,CTD,VIC,SAB,MWG,FPT,FRT
date,,,,,,,,,
2020-10-01,26.60,84.8,27.00,69.0,92.5,184.5,105.0,50.8,24.20
2020-10-02,26.95,84.0,26.70,68.7,94.0,184.5,104.0,50.3,24.10
2020-10-05,27.50,83.5,27.35,67.0,93.3,184.5,104.0,50.5,24.05
2020-10-06,27.20,83.9,28.25,64.5,91.8,186.3,105.2,50.5,24.45
2020-10-07,27.00,85.0,27.80,60.5,92.5,190.0,107.6,50.4,25.05
...,...,...,...,...,...,...,...,...,...
2023-08-30,32.10,89.1,27.20,61.7,62.0,159.8,53.8,96.6,83.30
2023-08-31,32.45,89.1,27.60,63.1,62.1,158.0,53.8,96.7,83.50
2023-09-05,32.65,90.1,27.80,65.3,62.3,158.8,54.3,96.8,82.60


In [ ]:
stocks_df_pivot.resample('M').last()


,CTG,VCB,HPG,CTD,VIC,SAB,MWG,FPT,FRT
date,,,,,,,,,
2020-10-31,29.00,83.0,30.55,56.40,106.5,184.3,104.70,51.5,20.85
2020-11-30,33.50,93.0,35.50,64.20,103.9,191.5,116.00,55.4,23.75
2020-12-31,34.55,97.9,41.45,77.70,108.2,195.0,118.90,59.1,32.00
2021-01-31,30.50,93.0,39.15,74.00,99.4,161.7,132.70,62.7,26.20
2021-02-28,37.25,98.2,45.60,73.90,109.0,188.0,135.00,76.2,28.00
2021-03-31,40.15,94.8,46.80,72.90,117.9,180.0,129.70,77.7,30.45
2021-04-30,40.80,100.0,58.10,64.20,131.0,167.1,140.90,80.9,27.00
2021-05-31,53.10,98.6,52.70,62.00,117.5,155.5,138.50,97.9,24.90
2021-06-30,52.70,116.4,51.50,64.70,119.0,169.1,152.00,88.0,27.10


In [1]:
import pandas as pd
from prophet import Prophet
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

Importing plotly failed. Interactive plots will not work.


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>